# This notebook will demonstrate all the features I developed for the project.

In [4]:
import db_utils as dbu

## Get all the connecting information

In [5]:
display(dbu.db_query_to_df('select * from record_albums'))
display(dbu.db_query_to_df('select * from record_artists'))
display(dbu.db_query_to_df('select * from group_members'))
display(dbu.db_query_to_df('select * from record_genres'))
display(dbu.db_query_to_df('select * from record_labels'))
display(dbu.db_query_to_df('select * from record_tracks'))
display(dbu.db_query_to_df('select * from record_sales'))

,album_name,album_release_date,artist_id,genre_id,record_label_id
album_id,,,,,
1,The Dark Side of the Moon,1973-03-01,1,1,1
2,Boston,1976-08-25,2,1,2


,artist_name
artist_id,
1,Pink Floyd
2,Boston


,member_name
member_id,
1,Syd Barrett
2,Roger Waters
3,Richard Wright
4,Nick Mason
5,David Gilmour
6,Tom Scholz
7,Brad Delp
8,Barry Goudreau
9,Fran Sheehan


,genre_name,genre_description
genre_id,,
1,Rock,Rock music


,record_label_name
record_label_id,
1,Harvest Capitol Records
2,Epic Records


,track_name,track_number,genre_id
track_id,,,
1,Speak to Me,1,1
2,Breathe (In the Air),2,1
3,On the Run,3,1
4,Time,4,1
5,The Great Gig in the Sky,5,1
6,Money,6,1
7,Us and Them,7,1
8,Any Colour You Like,8,1
9,Brain Damage,9,1


,album_id,sale_date,sale_quantity,unit_sale_price
sale_id,,,,
1,1,1973-01-01,1000000,10
2,1,1974-01-01,1000000,10
3,1,1975-01-01,1000000,10
4,1,1976-01-01,1000000,10
5,1,1977-01-01,1000000,10
6,1,1978-01-01,1000000,10
7,1,1979-01-01,1000000,10
8,1,1980-01-01,1000000,10
9,1,1981-01-01,1000000,10


## Read all the album information from the database view album_information into a dataframe

In [6]:
query = 'select * from album_information'
albums_df = dbu.db_query_to_df(query)
albums_df = albums_df.reset_index()
albums_df = albums_df.set_index(['album_name', 'album_release_date', 'artist_name'])
display(albums_df)

,,,genre_name,record_label_name
album_name,album_release_date,artist_name,,
The Dark Side of the Moon,1973-03-01,Pink Floyd,Rock,Harvest Capitol Records
Boston,1976-08-25,Boston,Rock,Epic Records


## Read all the album detail information from the database view album_information details into a dataframe

In [7]:
query = 'select * from album_information_details'
albums_df = dbu.db_query_to_df(query)
albums_df = albums_df.reset_index()
albums_df = albums_df.set_index(['album_name', 'album_release_date', 'artist_name', 'record_label_name', 'track_number'])
display(albums_df)

genre_name  \
album_name                album_release_date artist_name record_label_name       track_number              
Boston                    1976-08-25         Boston      Epic Records            1                  Rock   
                                                                                 2                  Rock   
                                                                                 3                  Rock   
                                                                                 4                  Rock   
                                                                                 5                  Rock   
                                                                                 6                  Rock   
                                                                                 7                  Rock   
                                                                                 8                  Rock   
The Dark Side of the Moon 1973-03-01         Pink Floyd  Harvest Capitol Records 1                  Rock   
                                                                                 2                  Rock   
                                                                                 3                  Rock   
                                                                                 4                  Rock   
                                                                                 5                  Rock   
                                                                                 6                  Rock   
                                                                                 7                  Rock   
                                                                                 8                  Rock   
                                                                                 9                  Rock   
                                                                                 10                 Rock   

                                                                                                                 track_name  
album_name                album_release_date artist_name record_label_name       track_number                                
Boston                    1976-08-25         Boston      Epic Records            1                      More Than a Feeling  
                                                                                 2                            Peace of Mind  
                                                                                 3                       Foreplay/Long Time  
                                                                                 4                         Rock & Roll Band  
                                                                                 5                                   Smokin  
                                                                                 6                             Hitch a Ride  
                                                                                 7                      Something About You  
                                                                                 8             Let Me Take You Home Tonight  
The Dark Side of the Moon 1973-03-01         Pink Floyd  Harvest Capitol Records 1                              Speak to Me  
                                                                                 2                     Breathe (In the Air)  
                                                                                 3                               On the Run  
                                                                                 4                                     Time  
                                                                                 5                 The Great Gig in the Sky  
                                           

## Show which members are in which bands

In [8]:
query = 'select * from band_members'
albums_df = dbu.db_query_to_df(query)
albums_df = albums_df.reset_index()
albums_df = albums_df.set_index(['artist_name'])
display(albums_df)

,member_name
artist_name,
Pink Floyd,Syd Barrett
Pink Floyd,Roger Waters
Pink Floyd,Richard Wright
Pink Floyd,Nick Mason
Pink Floyd,David Gilmour
Boston,Tom Scholz
Boston,Brad Delp
Boston,Barry Goudreau
Boston,Fran Sheehan
